## Classificador de frutas, legumes e verduras

Fruta: É um tipo de alimento derivado de plantas que geralmente é caracterizado por seu sabor doce, aroma distinto e elevado teor de suco. As frutas podem incluir frutos verdadeiros, como maçãs e bananas, bem como pseudofrutos, como morangos e tomates.<br /><br />

Legume: É uma categoria ampla de alimentos derivados de plantas que inclui frutos, talos, raízes e sementes. Os legumes são frequentemente consumidos como parte de uma dieta saudável e são ricos em vitaminas, minerais e outros nutrientes importantes.<br /><br />

Verdura: São vegetais que podem ser consumidos como alimento, incluindo folhas, flores, botões ou talos da planta. As verduras são uma fonte importante de vitaminas, minerais e fibras, e são frequentemente incluídas em dietas saudáveis para complementar a ingestão de outros nutrientes.<br /><br />

Frutas, legumes e verduras são tipos diferentes de alimentos vegetais que pertencem a diferentes categorias. Frutas são produtos da flor da planta, geralmente caracterizados por seu sabor adocicado e aroma distinto. Legumes incluem frutos, raízes, caules e sementes da planta que são utilizados na alimentação. Verduras são partes comestíveis da planta, como folhas, flores, botões ou hastes. Em resumo, frutas são as sementes envoltas em polpa adocicada, legumes são os vegetais comestíveis que não são frutos e verduras são os vegetais folhosos.

*Dados: https://www.kaggle.com/datasets/idhytm/dataset-frutas-legumes-e-verduras-pt-br

In [21]:
#@title libs
# !pip install pandas
# !pip install scipy
# !pip install sklearn
# !pip install pickle

In [22]:
import requests
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [23]:
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import graphviz
from sklearn.tree import export_graphviz

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
def upload_databases(name, url):
    response = requests.get(url)
    print(f">> db: {name}, response: {response};")
    text = response.content.decode("utf-8")
    return text

In [26]:
url_db_frutas = "https://docs.google.com/uc?export=download&id="
url_db_legumes = "https://docs.google.com/uc?export=download&id="
url_db_verduras = "https://docs.google.com/uc?export=download&id="

db_frutas = upload_databases("db_frutas", url_db_frutas)
db_legumes = upload_databases("db_legumes", url_db_legumes)
db_verduras = upload_databases("db_verduras", url_db_verduras)

>> db: db_frutas, response: <Response [200]>;
>> db: db_legumes, response: <Response [200]>;
>> db: db_verduras, response: <Response [200]>;


In [27]:
print("db_frutas: '{}...',\ndb_legumes: '{}...',\ndb_verduras: '{}...'".format(
    db_frutas[:55], db_legumes[:55], db_verduras[:55])
)


db_frutas: 'Abacate; Abacaxi; Abiu; Araçá; Azeitona; Açaí; Acerola;...',
db_legumes: 'Abobrinha; Abóbora; Aipim; Alho; Amaranto; Amêndoa; Ame...',
db_verduras: 'Açafrão; Acelga; Acelga-chinesa; Agave-caribenho; 
Agr...'


In [28]:
#@title processamento

def processador_de_texto(data):
    data = unidecode(data)
    data = data.replace("-", " ")
    data = data.replace("  ", " ")
    data = re.sub('[^\w\s]', '', str(data.lower()))
    data = data.replace("\r", " ")
    data = data.replace("\n", " ")
    data = data.replace("  ", " ")
    data = data.split(" ")
    return data

In [29]:
db_list = [
    ["Fruta", db_frutas], ["Legume", db_legumes], ["Verdura", db_verduras],
]

In [30]:
db_processed = [[db_list[i][0], i, processador_de_texto(db_list[i][1])] for i, _ in enumerate(db_list)]

In [31]:
print("> frutas:", len(db_processed[0][2]))
print("> legumes:", len(db_processed[1][2]))
print("> verduras:", len(db_processed[2][2]))

> frutas: 201
> legumes: 196
> verduras: 199


In [32]:
dataset_horticolas = pd.DataFrame(columns=["horticolas", "classe", "labels"])
for name, num, lista in db_processed:

    for i in lista:
        temp_df = pd.DataFrame(
            {'horticolas': [i], 'classe': [name], 'labels': [float(num)]}
        )
        dataset_horticolas = pd.concat([dataset_horticolas, temp_df], axis=0, ignore_index=True)


print(dataset_horticolas.shape)
display(dataset_horticolas.head())
print(dataset_horticolas["labels"].value_counts())

(596, 3)


,horticolas,classe,labels
0,abacate,Fruta,0.0
1,abacaxi,Fruta,0.0
2,abiu,Fruta,0.0
3,araca,Fruta,0.0
4,azeitona,Fruta,0.0


0.0    201
2.0    199
1.0    196
Name: labels, dtype: int64


In [33]:
#@title Tokenizer

def save_vector(vector):
    with open(f"hortalicia_vectorizer.pickle", "wb") as f:
        pickle.dump(vector, f)
    print(">> 'hortalicia_vectorizer.pickle' has been created")
    return

vectorizer = TfidfVectorizer()
myvectorizer = vectorizer.fit(dataset_horticolas["horticolas"])
save_vector(myvectorizer)

>> 'hortalicia_vectorizer.pickle' has been created


In [34]:
X = myvectorizer.fit_transform(dataset_horticolas["horticolas"])
y = dataset_horticolas["labels"]

In [35]:
#@title Model

def generate_machine_learning_m(X, y):
    
    model = RandomForestClassifier()

    model.fit(X, y)

    result_y = model.score(X, y); print(f">> model precicion: {result_y:.2%}")
    return model

def downlaod_model(model):
    with open("hortalicia_model.pkl", "wb") as f:
        pickle.dump(model, f)
    print(">> 'hortalicia_model' has been downloaded!")
    return

In [36]:
model = generate_machine_learning_m(X, y)
downlaod_model(model)

>> model precicion: 83.39%
>> 'hortalicia_model' has been downloaded!


In [37]:
def classify_new(input_, myvectorizer):

    input_vect = myvectorizer.transform(input_)

    input_vect_array = np.array(input_vect)
    input_vect_array_csr_matrix = csr_matrix((input_vect_array).all())

    return input_vect_array_csr_matrix

In [38]:
def input_validation(query, myvectorizer):
    print(query)
    print(type(query))
    
    # classify_new(processador_de_texto(classify), myvectorizer)
    return

In [41]:
#@title Classificar hortícola

classify = input("Digite um hortícola: ")

classify_vector = classify_new(processador_de_texto(classify), myvectorizer)

print(f">> input vectorized: '{classify_vector}'")

input_validation = True

if not str(classify_vector):
    input_validation = False

if input_validation:
    proba = model.predict_proba(classify_vector)
    classification = model.predict(classify_vector)

    src_class = dataset_horticolas.loc[ dataset_horticolas["labels"] == classification[0]]

    print(f">> probabilidades: '{proba[0]}'")

    print(">> '{}' ({}) seria a classificação correta para o hortícola '{}'!".format(
        src_class["classe"].values[0], classification[0], classify)
    )
else:
    print(f"O pedido para '{classify}' foi negado!")

>> input vectorized: '  (0, 93)	1.0
  (1, 128)	1.0
  (2, 287)	1.0'
>> probabilidades: '[0. 1. 0.]'
>> 'Legume' (1.0) seria a classificação correta para o hortícola 'Castanha-do-para'!
